In [1]:
PROJECT_NAME = "square_bar_llama_2_70B"

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import Replicate
import os
from dotenv import load_dotenv
load_dotenv()
replicate_api_key = os.environ["REPLICATE_API_TOKEN"]
template = """You are a helpful assistant for creating a .sif file for use in multiphysics simulations with the ELMER software. Analyze the previous geometry and the simulation requirements the user wants, and ask questions on all required sections explained below until you have all the data to produce the file. When writing the file follow these sections meticulously:

Header Section: (Automatically include this section in the .sif file.)
Header
CHECK KEYWORDS Warn
Mesh DB "." "."
Include Path ""
Results Directory ""
End

Simulation Details:

Specify if the simulation type is "Steady state" or "Transient".
Constants Section: (Automatically include this section.)
Constants
...
End

Body Properties: (Refer to the physical groups defined in the .geo file.)
Body 1
...
End

Solver Configuration: (Automatically include this section.)
Solver 1
...
End

Equation Settings: (Automatically include this section.)
Equation 1
...
End

Material Properties:

Reference the named physical groups from the .geo file for material assignments.
Initial Conditions: (Automatically include this section.)
Initial Condition 1
...
End

Boundary Conditions:

Based on the geometry defined in the .geo file, specify boundary conditions for each named physical group.
Output Instructions:

Produce only the final text of the .sif file.
Enclose the entire .sif file content within specific markers:
Start the file content with //BEGIN_SIF.
End the file content with //END_SIF.
Ensure that the text between these markers is exactly as it should appear in the .sif file, with correct syntax and formatting.
Example:
//BEGIN_SIF
[Insert the complete and final .sif file content here]
//END_SIF"

I provide an example of sif file for linear elesticity from the ELMER tutorials
!ElmerSolver input file from ElmerFront
!Saved        = Wed May 14 09:51:11 2003 User=apursula Host=hiisi.csc.fi
!Case         = Beam  
!Model dir    = /mnt/hiisi/wrk/apursula/tutorialfiles/elast_beam/linear
!Include path = 
!Results dir  = /mnt/hiisi/wrk/apursula/tutorialfiles/elast_beam/linear

!Bodies 1
!Equations 1
!Solvers 1
!Materials 1
!Body Forces 0
!Initial Conditions 0
!Boundary Conditions 2
!Boundaries 4

!echo on

Header
  CHECK KEYWORDS Warn
  Mesh DB "MESHDIR" "mesh1"
  Include Path ""
  Results Directory ""
End

Simulation
  Min Output Level = 0
  Max Output Level = 31
  Output Caller = True

  Coordinate System = "Cartesian 2D"
  Coordinate Mapping(3) = 1 2 3

  Simulation Type = "Steady State"
  Steady State Max Iterations = 20
  Output Intervals = 1

  Solver Input File = "Beam.sif"
  Output File = "Beam.dat"
  Post File = "Beam.ep"
  Mesh Input File
    File "Beam.mif"

End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.67e-08
End

Body 1
  Name = "Body1"

  Equation = 1
  Material = 1
End

Equation 1
  Name = "Equation1"

  Stress Analysis = True
  Calculate Stresses = Logical False
End

Solver 1
  Exec Solver = "Always"
  Equation = "Stress Analysis"
  Variable = "Displacement"
  Variable Dofs = 2
  Linear System Solver = "Iterative"
  Linear System Iterative Method = "BiCGStab"
  Linear System Max Iterations = 300
  Linear System Convergence Tolerance = 1.0e-08
  Linear System Abort Not Converged = True
  Linear System Preconditioning = "ILU0"
  Linear System Residual Output = 1
  Steady State Convergence Tolerance = 1.0e-05
  Nonlinear System Convergence Tolerance = 1.0e-05
  Nonlinear System Max Iterations = 1
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-02
  Nonlinear System Relaxation Factor = 1.0
  Adaptive Error Limit = 0.1
  Adaptive Max Change = 2
  Adaptive Coarsening = Logical True
  Time Derivative Order = 2
End

Material 1
  Name = "Material1"

  Youngs Modulus = 200e9
  Poisson Ratio = 0.3
End

Boundary Condition 1
  Name = "Constraint1"
  Target Boundaries(1) = 4 

  Displacement 1 = 0
  Displacement 2 = 0
End


Boundary Condition 2
  Name = "Constraint2"
  Target Boundaries(1) = 3 

  Force 2 = Variable Coordinate 1
    Real
      0 0
      1 -1.0000e+07
    End
End

!End Of File

Here another example
Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
End

Simulation
  Max Output Level = 1
  Coordinate System = Cartesian
  Coordinate Mapping(3) = 1 2 3
  Simulation Type = Steady state
  Steady State Max Iterations = 1
  Output Intervals(1) = 1
  Solver Input File = case.sif
  Post File = case.vtu
End

Constants
  Gravity(4) = 0 -1 0 9.82
  Stefan Boltzmann = 5.670374419e-08
  Permittivity of Vacuum = 8.85418781e-12
  Permeability of Vacuum = 1.25663706e-6
  Boltzmann Constant = 1.380649e-23
  Unit Charge = 1.6021766e-19
End

Body 1
  Target Bodies(1) = 1
  Name = "Body Property 1"
  Equation = 1
  Material = 1
  Initial condition = 1
End

Solver 1
  Equation = Linear elasticity
  Procedure = "StressSolve" "StressSolver"
  Exec Solver = Always
  Stabilize = True
  Optimize Bandwidth = True
  Steady State Convergence Tolerance = 1.0e-5
  Nonlinear System Convergence Tolerance = 1.0e-7
  Nonlinear System Max Iterations = 20
  Nonlinear System Newton After Iterations = 3
  Nonlinear System Newton After Tolerance = 1.0e-3
  Nonlinear System Relaxation Factor = 1
  Linear System Solver = Iterative
  Linear System Iterative Method = BiCGStab
  Linear System Max Iterations = 500
  Linear System Convergence Tolerance = 1.0e-10
  BiCGstabl polynomial degree = 2
  Linear System Preconditioning = ILU0
  Linear System ILUT Tolerance = 1.0e-3
  Linear System Abort Not Converged = False
  Linear System Residual Output = 10
  Linear System Precondition Recompute = 1
End

Equation 1
  Name = "elasticity"
  Active Solvers(1) = 1
End

Material 1
  Name = "Steel (stainless - generic)"
  Heat expansion Coefficient = 14.9e-6
  Youngs modulus = 200.0e9
  Density = 7925.0
  Heat Conductivity = 24.0
  Mesh Poisson ratio = 0.285
  Heat Capacity = 460.0
  Sound speed = 5100.0
  Poisson ratio = 0.3
End

Initial Condition 1
  Name = "InitialCondition"
  Displacement 1 = 0
  Displacement Velocity 1 = 0
  Displacement Velocity 2 = 0
  Displacement Velocity 3 = 0
  Displacement 3 = 0
  Displacement 2 = 0
End

Boundary Condition 1
  Target Boundaries(1) = 2 
  Name = "fixed"
  Displacement 1 = 0
  Displacement 2 = 0
  Displacement 3 = 0
End

Boundary Condition 2
  Target Boundaries(1) = 1 
  Name = "subjected_to_force"
  Force 2 = 200000000
  Force 1 = 0
  Force 3 = 0
End

There is no Output section in the sif file

If unclear, ask which surfaces correspond to a specific boundary condition.
To help you understanding the geometry of the system, below you will find the text of the .geo file with geometry. 
geo file:// Define the dimensions
L = 10; // Length of the bar in cm
a = 1; // Side length of the square cross-section in cm

// Define the points
Point(1) = {{0, 0, 0, a}}; // Bottom left
Point(2) = {{a, 0, 0, a}}; // Bottom right
Point(3) = {{a, a, 0, a}}; // Top right
Point(4) = {{0, a, 0, a}}; // Top left
Point(5) = {{0, 0, L, a}}; // Top left on the top surface
Point(6) = {{a, 0, L, a}}; // Top right on the top surface
Point(7) = {{a, a, L, a}}; // Bottom right on the top surface
Point(8) = {{0, a, L, a}}; // Bottom left on the top surface

// Define the lines
Line(1) = {{1, 2}}; // Bottom edge
Line(2) = {{2, 3}}; // Right edge
Line(3) = {{3, 4}}; // Top edge
Line(4) = {{4, 1}}; // Left edge
Line(5) = {{5, 6}}; // Top edge on the top surface
Line(6) = {{6, 7}}; // Right edge on the top surface
Line(7) = {{7, 8}}; // Bottom edge on the top surface
Line(8) = {{8, 5}}; // Left edge on the top surface
Line(9) = {{1, 5}}; // Vertical edge 1
Line(10) = {{2, 6}}; // Vertical edge 2
Line(11) = {{3, 7}}; // Vertical edge 3
Line(12) = {{4, 8}}; // Vertical edge 4

// Define the surfaces
Line Loop(1) = {{1, 2, 3, 4}}; // Bottom surface
Plane Surface(1) = {{1}}; // Bottom surface
Line Loop(2) = {{5, 6, 7, 8}}; // Top surface
Plane Surface(2) = {{2}}; // Top surface
Line Loop(3) = {{1, 10, -5, -9}}; // Front surface
Plane Surface(3) = {{3}}; // Front surface
Line Loop(4) = {{2, 11, -6, -10}}; // Right surface
Plane Surface(4) = {{4}}; // Right surface
Line Loop(5) = {{3, 12, -7, -11}}; // Back surface
Plane Surface(5) = {{5}}; // Back surface
Line Loop(6) = {{4, 9, -8, -12}}; // Left surface
Plane Surface(6) = {{6}}; // Left surface

// Define the volume
Surface Loop(1) = {{1, 2, 3, 4, 5, 6}}; // Enclosing surfaces
Volume(1) = {{1}}; // The bar volume

// Define physical groups for boundaries
Physical Surface("Bottom") = {{1}}; // Bottom surface
Physical Surface("Top") = {{2}}; // Top surface
Physical Surface("Front") = {{3}}; // Front surface
Physical Surface("Right") = {{4}}; // Right surface
Physical Surface("Back") = {{5}}; // Back surface
Physical Surface("Left") = {{6}}; // Left surface
Physical Volume("Bar") = {{1}}; // The bar volume

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)     

In [3]:
#Saves .geo file from response
def extract_and_save_geo_file(response_text,PROJECT_NAME=PROJECT_NAME): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [4]:
#Saves .geo file from response
def extract_and_save_sif_file(response_text): 
    file_path = f"{PROJECT_NAME}.sif"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_SIF"
        end_marker = f"//END_SIF"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")
        
        
        
        
import os

def generate_ELMER_mesh():
    try:
        current_directory = os.getcwd()
        msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        !"C:\Program Files\ElmerFEM-nogui-nompi-Windows-AMD64\bin\ElmerGrid.exe" 14 2 "{msh_file}" -autoclean
        print(f"ELMER project created in {PROJECT_NAME}")
        print(f"Copy .sif file in {PROJECT_NAME}")
        
    except ValueError as e:
        print(f"ERROR.")

In [5]:
llama = Replicate(model="meta/llama-2-70b-chat")

In [6]:
chain = LLMChain(
    llm=llama,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_tokens":4096,"temperature": 0.00}
)

C:\Users\ossama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
generate_ELMER_mesh()

ELMER project created in square_bar_llama_2_70B
Copy .sif file in square_bar_llama_2_70B

Starting program Elmergrid, compiled on May  3 2024
Elmergrid reading in-line arguments
Lower dimensional boundaries will be removed
Materials and boundaries will be renumbered
Nodes that do not appear in any element will be removed
Output will be saved to file c:\Users\ossama\Desktop\LLM_LIMITS\SIMULATION\LLAMA\LLAMA_2_70B\square_bar_llama_2_70B.

Elmergrid loading data:
-----------------------
Format chosen using the first line: $MeshFormat
Gmsh version is 4.1
Loading mesh in Gmsh format 4.1 from file c:\Users\ossama\Desktop\LLM_LIMITS\SIMULATION\LLAMA\LLAMA_2_70B\square_bar_llama_2_70B.msh
Reading 8 entities in 0D
Reading 12 entities in 1D
Reading 6 entities in 2D
Reading 1 entities in 3D
Allocating for 10250 knots and 57462 elements.
Leading element dimension is 3
BC name for physical group 1 is: Bottom
BC name for physical group 2 is: Top
BC name for physical group 3 is: Front
BC name for phy

In [8]:
output = chain.predict(
    human_input="""Create a sif file for a 'linear elasticity' simulation of a steel bar with the geometry in the geo file. 
Boundary 1 is fixed (no deformation), while Boudary 2 is subjected of a Force of 100000000 N in the y direction""")
print(output)

 Sure, I can help you with that. Based on the geometry defined in the .geo file, I understand that we have a steel bar with a square cross-section and a length of 10 cm. The bar is fixed at one end (Boundary 1) and subjected to a force of 100000000 N in the y direction at the other end (Boundary 2).

To create a sif file for this simulation, we need to define the following sections:

1. Header: This section includes general information about the simulation, such as the simulation


In [9]:
output = chain.predict(
    human_input="""
Simulation Type: "Steady state",
Constants: do not include gravity,
Material Properties: use typical values for steel,
Boundary Conditions: "Boundary 1" is Bottom and "Boundary 2" the "Top" in the .geo file
""")
print(output)

 Sure, I can help you with that. Here's a possible sif file for the simulation you described:

//BEGIN_SIF

Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
  Simulation Type Steady state
  Constants
    Gravity(4) = 0 -1 0 9.82
    Stefan Boltzmann = 5.670374419e-08
    Permittivity of Vacuum = 


In [11]:
output = chain.predict(human_input = "you did not complete your output...")
print(output)

 I apologize for the incomplete response. Here is the complete sif file based on the information provided:

//BEGIN_SIF

Header
  CHECK KEYWORDS Warn
  Mesh DB "." "."
  Include Path ""
  Results Directory ""
  Simulation Type Steady state
  Constants
    Gravity(4) = 0 -1 0 9.82
    Stefan Boltzmann = 5.670374419e-08
    Permittivity of Vacuum = 8.


In [ ]:
extract_and_save_sif_file(output)

In [ ]:
import os
new_directory = os.path.join(os.getcwd(), f"{PROJECT_NAME}")
os.chdir(new_directory)

In [ ]:
pwd

In [ ]:
!"C:\Program Files\ElmerFEM-nogui-nompi-Windows-AMD64\bin\ElmerSolver.exe" "C:\Users\ossama\Desktop\LLM_LIMITS\SIMULATION\LLAMA\LLAMA_3_8B\SQUARE_BAR\square_bar_llama_3_8B.sif"